In [1]:
!pip install baostock

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     --------------------------------------- 55.2/55.2 kB 39.4 kB/s eta 0:00:00


In [2]:
import baostock as bs
import pandas as pd

def get_stockdata(code):
    rs = bs.query_history_k_data_plus(code,
                                      'date,close,volume,turn',
                                      start_date='2020-01-01',
                                      end_date='2022-01-01',
                                      frequency='d', adjustflag='2') 

    return rs.get_data()

In [3]:
prediction_length = 20
stock_list = ['sh.600227', 'sh.600200', 'sh.600201']
train_dic_list = []
test_dic_list = []
lg = bs.login()

for stock_id in stock_list:
    df = get_stockdata(stock_id)
    train_dic = {'start':df.date[0],
                 'target':df.close,
                 'cat':int(stock_id.split('.')[1]),
                 'dynamic_feat':[df.volume, df.turn]}
    
    test_dic = {'start':df.date[0],
                'target':df.close[:-prediction_length],
                'cat':int(stock_id.split('.')[1]),
                'dynamic_feat':[df.volume[:-prediction_length], df.turn[:-prediction_length]]}
    train_dic_list.append(train_dic)
    test_dic_list.append(test_dic)

bs.logout()

login success!
logout success!


In [8]:
train_dic_list[0]['dynamic_feat']

[0       8267069
 1      42399330
 2      41427418
 3      23965472
 4      20859994
          ...   
 481    27560861
 482    19039600
 483    50192183
 484    29448496
 485    32671104
 Name: volume, Length: 486, dtype: object,
 0      0.646800
 1      3.317200
 2      3.241100
 3      1.875000
 4      1.632000
          ...   
 481    2.156300
 482    1.489600
 483    3.926800
 484    2.303900
 485    2.556100
 Name: turn, Length: 486, dtype: object]

In [9]:
estimator = DeepAREstimator(
    prediction_length=prediction_length,
    context_length=60,
    freq='1d',
    num_layers=2,
    num_cells=64,
    trainer=Trainer(epochs=20,
                    learning_rate=1e-2,
                    num_batches_per_epoch=32)
)
predictor = estimator.train(train_data)

NameError: name 'DeepAREstimator' is not defined